In [10]:
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from credentials import CLIENT_ID, CLIENT_SECRET

In [2]:
SPOTIPY_CLIENT_ID= CLIENT_ID
SPOTIPY_CLIENT_SECRET= CLIENT_SECRET
SPOTIPY_REDIRECT_URI= "http://127.0.0.1:5500/"
SCOPE = "user-top-read"

In [3]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=SPOTIPY_CLIENT_ID, 
    client_secret=SPOTIPY_CLIENT_SECRET, 
    redirect_uri=SPOTIPY_REDIRECT_URI, 
    scope=SCOPE)
    )

In [4]:
results = sp.current_user_top_tracks(limit= 50, time_range="short_term")
track_ids = [song['id'] for song in results["items"]]
all_track_info = {}

for id in track_ids:
    meta = sp.track(id)
    track_info = {}
    track_info['name'] = meta['name']
    track_info['album'] = meta['album']['name']
    track_info['artist'] = meta['album']['artists'][0]['name']
    track_info['url'] = meta['external_urls']['spotify']
    track_info['album_cover'] = meta['album']['images'][0]['url']
    all_track_info[id] = track_info

In [15]:
import json
with open('lilian_spotify.json', 'w') as f:
    json.dump(all_track_info, f)

In [19]:
import json
with open('han_spotify.json') as f:
    han_track_ids = json.load(f)

# Insights

In [29]:
def unique_tracks(track_id_list):
    track_ID_count = np.unique(track_id_list, return_counts=True)
    return track_ID_count[0][np.where(track_ID_count[1]>1)]

In [49]:
def artists(track_id_list):
    all_track_artists = []
    for lst in track_id_list:
         all_track_artists.append(np.unique([lst[key]['artist'] for key in lst]))
    all_artist_count = np.unique(np.concatenate(all_track_artists), return_counts=True)
    return all_artist_count[0][np.where(all_artist_count[1]>1)]

In [66]:
def audio_features(track_id_list):
    features = ["danceability", "energy", "liveness", "loudness", "speechiness", "valence", "instrumentalness"]

    track_audio_feature = {}
    for track_list in track_id_list:
        for ID in track_list:
            track_feature = sp.audio_features(ID)[0]
            for feature in features:
                track_audio_feature[feature] =  track_audio_feature.get(feature,0) + track_feature[feature]

    for feature in features:
        track_audio_feature[feature] = track_audio_feature.get(feature)/(len(track_id_list)*len(track_id_list[0]))
    
    return track_audio_feature

In [30]:
all_unique_tracks = unique_tracks(list(all_track_info.keys())+list(han_track_ids.keys()))

In [70]:
artists([all_track_info, han_track_ids])

In [67]:
all_track_audio_features = audio_features([all_track_info, han_track_ids])

In [ ]:
if 'liveness'>0.8:
    print("You listen to a lot of live music!")

if 'energy' >0.5:
    print("Your music is really energetic!")

if 'danceability'>0.5:
    print("Have you been having dance parties?")

if 'valence' <= 0.5:
    print("Looks like your music taste is on the sad side. Do you need a pick-me-up?")
else:
    print("Your music says you've been in a good mood this week!")

if 'instrumentalness' >=0.5:
    print("You've got instrumental vibes going on this week.")
